In [1]:
"hello world"

'hello world'

In [38]:
from langchain_core.tools import tool
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated


In [67]:

@tool
def get_conversion_factor(base_currency:str, target_currecy:str)->float:
    """This function fetches the currency conversion factor between a given base currency and a target currency"""

    url=f"https://v6.exchangerate-api.com/v6/a0e25f3dfacaebb37092c41d/pair/{base_currency}/{target_currecy}"
    response=requests.get(url)
    return response.json()["conversion_rate"]

@tool
def convert(base_currency_value:int, exchange_factor: Annotated[float, InjectedToolArg])->float:
    """
   given a currency conversion rate this function calculates the target currency value from a given base currency value
    """
    return base_currency_value*exchange_factor



In [68]:
get_conversion_factor.invoke({"base_currency":"USD","target_currecy":"INR"})
convert.invoke({"base_currency_value":10,"exchange_factor":91.524})

915.24

In [60]:
llm = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", temperature=0)


In [61]:
llm_with_tools=llm.bind_tools([get_conversion_factor,convert])

In [62]:
messages=[HumanMessage("what is the conversion factor between USD and INR ,and based on that can you convert 10 USD to INR")]

In [63]:
messages

[HumanMessage(content='what is the conversion factor between USD and INR ,and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={})]

In [69]:
ai_mess=llm_with_tools.invoke(messages)

In [70]:
ai_mess.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currecy': 'INR', 'base_currency': 'USD'},
  'id': 'a6502f7d-9476-4096-8158-b3911be3596b',
  'type': 'tool_call'}]

In [73]:
for tool_call in ai_mess.tool_calls:
    if tool_call["name"]=="get_conversion_factor":
        tool_mess_1=get_conversion_factor.invoke(tool_call)
        print(tool_mess_1)

content='91.524' name='get_conversion_factor' tool_call_id='a6502f7d-9476-4096-8158-b3911be3596b'
